# Monitor CPC Alarm Messages

## Setup

### Import Libraries

In [ ]:
# Standard Library
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pathlib import Path
import smtplib
import sys
#3rd Party
import pandas as pd
from sqlalchemy import create_engine, text
# Local
base = Path().resolve().parents[2]
sys.path.insert(0, str(base))
from shared_config.src.shared_config.sql import PYRO_SERVER, PROD_SERVER, DEV_SERVER, PYRO_DATABASE
from shared_utils.src.shared_utils.sql import get_engine, read_table, write_to_sql

pd.set_option('display.max_rows', None)

print("Initializing Python script...\n")
print("Importing libraries...\n")

### Connect to Pyrometry Database on DEV/QUAL/PROD Server

In [ ]:
dev_engine = get_engine(DEV_SERVER, PYRO_DATABASE)
prod_engine = get_engine(PROD_SERVER, PYRO_DATABASE)

recent_message_id = read_table(dev_engine, 'SELECT * FROM [CPC_Email_Reference]')
ignore_alarms = read_table(dev_engine, 'SELECT * FROM [CPC_Email_IgnoreAlarms]')
email_list = read_table(dev_engine, 'SELECT * FROM [CPC_Email_EmailList]')

### SQL Server Database Connection

In [ ]:
print("Connecting to Pyrometry Server databases...\n")

pyro_engine = get_engine(PYRO_SERVER, 'master')

query_get_databases = """
    SELECT name 
    FROM sys.databases 
    WHERE name LIKE 'CPC_%' 
        AND name NOT LIKE 'CPC_Staging'
    """
df_messages = []

try:
    with pyro_engine.connect() as connection:
        databases = [row[0] for row in connection.execute(text(query_get_databases))]

    for db_name in databases:
        db_conn_str = f'mssql+pyodbc://{PYRO_SERVER}/{db_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
        db_engine = create_engine(db_conn_str)

        with db_engine.connect() as db_connection:
            try:
                query_get_messages = """
                    SELECT TOP 200
                        *
                    FROM 
                        dbo.RealtimeMessages
                    WHERE
                        MessageType = 'Alarm'
                    ORDER BY MessageTime DESC 
                """
                result = db_connection.execute(text(query_get_messages))
                
                df = pd.DataFrame(result.fetchall(), columns=result.keys())
                if not df.empty:
                    df_messages.append(df)
            except Exception as e:
                print(f'Error: {e}')
                print(f'Query: {query_get_messages}')
                raise e
except Exception as e:
    print(f'Error: {e}')

if df_messages:
    combined_df = pd.concat(df_messages, ignore_index=True)



In [ ]:
max_message_id = combined_df.groupby('Equipment', as_index=False)['MessageID'].max()

In [ ]:
merged_df = combined_df.merge(recent_message_id, on='Equipment', how='left', suffixes=('', '_max'))
merged_df['MessageID'] = merged_df['MessageID'].fillna(0)
merged_df = merged_df.merge(email_list, on='Equipment', how='left')

filtered_df = merged_df[merged_df['MessageID'] > merged_df['MessageID_max']]

In [ ]:
sorted_df = filtered_df.sort_values(by=['Equipment', 'MessageTime'], ascending=[True, False]).reset_index(drop=True)

In [ ]:
def ignore_alarm(row, ignore_df):
    equip = row['Equipment']
    text_lower = row['MessageText'].lower()

    for _, to_ignore in ignore_df.iterrows():
        ignore_equip = to_ignore['Equipment']
        phrase = to_ignore['IgnoreAlarms'].lower()
        if ignore_equip.lower() == 'all':
            if phrase in text_lower:
                return True
        else:
            if equip == ignore_equip and phrase in text_lower:
                return True
        
    return False

In [ ]:
final_df = sorted_df[~sorted_df.apply(lambda row: ignore_alarm(row, ignore_alarms), axis=1)].reset_index(drop=True)
final_df.sort_values(by='MessageTime', ascending=False)

### Set Email Information

In [ ]:
def send_email(df):
    # Email setup
    smtp_server = 'Relaywest.northgrum.com'
    smtp_port = 25
    sender_email = 'cpc_alarms@ngc.com'
    subject = 'TEST - Alarm Summary'
    
    # Email lists
    to_list = df['To_List'].iloc[0].split(',')
    # cc_list = df['Cc_List'].iloc[0].split(',')
    
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ', '.join(to_list)
    # msg['CC'] = ', '.join(cc_list)
    msg['Subject'] = subject

    # Email body
    body = f'''
    <html>
    <head>
        <title>TEST - Alarm Summary</title>
    </head>
    <body>
        <h2>TEST - Summary Email for the New Alarm Email System</h2>
        <p>The following critical alarms have recently triggered:</p>
        <table style="margin-left: 20px;">
            <tbody>
                {df[['Equipment', 'Site', 'MessageTime', 'MessageText', 'MessageType', 'Value', 'Criteria', 'Setting']].to_html(index=False)}
            </tbody>
        </table>
        <br>
        Thanks!,
        <br>
        CPC
    </body>
    </html>
    '''
    msg.attach(MIMEText(body, 'html'))
    
    # Send the email
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.sendmail(sender_email, to_list, msg.as_string())
            print('Email sent successfully!')
    except Exception as e:
        print(f'Email failed to send: {e}')

In [ ]:
if not final_df.empty:
    send_email(final_df)
else:
    print('No alarms to send.')

## Write to SQL Express Database

In [ ]:
write_to_sql(max_message_id, 'CPC_Email_Reference', dev_engine)